In [1]:
from bspump.jupyter import *
import random
import bspump.file
import time
from bspump.trigger import PeriodicTrigger
from datetime import datetime

In [2]:
new_pipeline("ExamplePipeline")

In [3]:
class RandomNumberSource(bspump.abc.source.TriggerSource):
  async def cycle(self):
    # Roll a die and run if the result is 1
    if random.randint(1, 6) == 1:
      await self.process(random.randint(1, 100))

In [4]:
@register_source
def init_rns(app, pipeline):
    return RandomNumberSource(app, pipeline).on(PeriodicTrigger(app, interval=0.05))

In [5]:
test_events([1, 2, 3])

In [6]:
@step
def enrich_event(event):
        current_time = datetime.now()
        timestamp = current_time.strftime('%Y-%m-%d %H:%M:%S.%f')
        # sleep a random amount of time between 1 and 100ms
        time.sleep(random.randint(1, 100) / 1000)

        event = {
            "timestamp": timestamp,
            "random_number": event,
            "doubled": event * 2,
        }
        return event

{'timestamp': '2023-11-20 22:56:35.619881', 'random_number': 1, 'doubled': 2}
{'timestamp': '2023-11-20 22:56:35.693050', 'random_number': 2, 'doubled': 4}
{'timestamp': '2023-11-20 22:56:35.714197', 'random_number': 3, 'doubled': 6}


In [7]:
@step
def add_squared(event):
    event["squared"] = event["random_number"] ** 2
    return event

{'timestamp': '2023-11-20 22:56:35.619881', 'random_number': 1, 'doubled': 2, 'squared': 1}
{'timestamp': '2023-11-20 22:56:35.693050', 'random_number': 2, 'doubled': 4, 'squared': 4}
{'timestamp': '2023-11-20 22:56:35.714197', 'random_number': 3, 'doubled': 6, 'squared': 9}


In [8]:
@step
def add_halved(event):
    event["halved"] = event["random_number"] / 2
    return event

{'timestamp': '2023-11-20 22:56:35.619881', 'random_number': 1, 'doubled': 2, 'squared': 1, 'halved': 0.5}
{'timestamp': '2023-11-20 22:56:35.693050', 'random_number': 2, 'doubled': 4, 'squared': 4, 'halved': 1.0}
{'timestamp': '2023-11-20 22:56:35.714197', 'random_number': 3, 'doubled': 6, 'squared': 9, 'halved': 1.5}


In [9]:
@register_sink
def init_file_sink(app, pipeline):
    return bspump.file.FileCSVSink(app, pipeline)

In [10]:
end_pipeline()